In [31]:
import requests
import os
import numpy as np
import pathlib


import tensorflow as tf
import cv2        as cv
import numpy      as np

from fastapi                  import FastAPI,File, UploadFile
from fastapi.middleware.cors  import CORSMiddleware
from mushroom_learning.gcp    import get_model

from tensorflow               import keras
from tensorflow.keras         import utils


# Checking API

In [38]:
URL_base    = 'http://127.0.0.1:8000/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"

## Check base URL

In [39]:
response = requests.get(URL_base)
print(response.json())

["Don't eat that mushroom!"]


## Check image size

In [81]:
image_path='../raw_data/Six_Species_Clean/10252_Amanita_muscaria/YE2013PIC18948304.JPG'
files = {'mush': image_path  }

In [82]:
response = requests.get(URL_size,data=files)
print(response.json())

This file is 0.074 Kbytes


## Check prediction

In [83]:
files = {'mush': image_path}
response = requests.get(URL_predict,data=files)
print(response.json())

This mushroom is most likely poisonous. Score: 0.64


### Check prediction against local model.

In [87]:
def image_API(im_path,size):
    # convert to bites (this step should be excluded in an actual API, as the
    # image arrives in bytes from the get or post functions.)
    with open(im_path, 'rb') as f:
        im_API = f.read()
    # convert to np array.    
    im_API=np.asarray(bytearray(im_API), dtype="uint8")
    # decode byte array back into image, and then adjust
    # for cv's automatic BGR representation
    im_API = cv.imdecode(im_API,cv.IMREAD_COLOR)
    im_API = cv.cvtColor(im_API , cv.COLOR_BGR2RGB)
    
    # resize using tensor flow with nearest neighbour interpolation
    im_API=tf.image.resize(im_API,size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # Expand for CNN
    im_API = tf.expand_dims(im_API, 0) 
    
    return im_API

In [94]:
im_API=image_API(image_path,(224,224))
model=get_model()

../our_first_model


In [96]:
prediction_API=model.predict(im_API)
classif_API = int(prediction_API > .5)
print(f'{prediction_API[0][0]:.5f}:{("poison","edible")[classif_API]}')

0.63804:edible
